# Imports

The data from Riot is accessed through riotwatcher. I imported ApiError as well so I can create some blocks later on that look for if I get an error code and figure why and what to do next. The data from the api is returned as a json file, that I will go through. I imported time so that I can make a timer between API calls due to the limits placed by Riot. I also need to load my api keys from an environment file, so that I don't make my API keys public :)

In [1]:
import pandas as pd
import json
import time

from riotwatcher import RiotWatcher
from riotwatcher import TftWatcher
from riotwatcher import ApiError

import os
from dotenv import load_dotenv

## Setting up Riot API access

Riot API key expires every 24 hours, so need to put new API key here for requests. Website to generate a new key: https://developer.riotgames.com

In [2]:
# Set the new API Key
load_dotenv()
API_KEY = os.getenv("RIOT_API_KEY")

# Riot ID
RIOT_ID = os.getenv("MY_RIOT_ID")

# Account Region
MY_REGION = os.getenv("MY_REGION")

# Account Server
MY_SERVER = os.getenv("MY_SERVER")

# Check if API Key works and if data is ready to pull
try: 
    account_watcher = RiotWatcher(API_KEY)
    tft_watcher = TftWatcher(API_KEY)
    print("TFT Watcher is ready to go!")
except Exception as e:
    print(f"Failed to initialize TftWatcher. Check your API Key. Error: {e}")
    watcher = None

TFT Watcher is ready to go!


## Script to get the data of all my games so far

Since this is the first time making this database, I will need to pull the data from all the games I have played already so far this set. In my case, I have 110 games played this set. So for the first run, I will set it to pull all my games, but after, it will refresh to grab the most recent 20 games.

In [3]:
# Define the path where the data will be stored
output_folder_path = os.path.join("DataFolders", "APIData")

# Define the base name for initial data file
output_filename_base = "InitialData"

# Creates the directory if it doesn't already exist
os.makedirs(output_folder_path, exist_ok=True)

# Checking the file path is correct
print(f"Data will be saved in: {output_folder_path}")

Data will be saved in: DataFolders/APIData


Riot saves each user data with a unique ID, so I need to use get my unique data using the account information from earlier

In [4]:
try:
    print("Getting your PUUID...")
    account_response = account_watcher.account.by_riot_id(MY_REGION, RIOT_ID.split('#')[0], RIOT_ID.split('#')[1])
    my_puuid = account_response['puuid']
    print(f"PUUID found!")

except ApiError as err:
    print(f"An API error occurred: {err}")

Getting your PUUID...
PUUID found!


Now to pull the match ids from the games of this set so I can start pulling the match data per game from the API.

In [6]:
try:
    print("Getting recent match IDs...")
    match_ids = tft_watcher.match.by_puuid(MY_REGION, my_puuid, count=120)
    print(f"Found {len(match_ids)} match IDs.")

except ApiError as err:
    print(f"An API error occurred: {err}")

Getting recent match IDs...
Found 120 match IDs.


Now I have all the match id's from my past 120 games. However, I just realized that pulling the recent 120 doesn't specify the game mode specifically, and I wanted just ranked games. Some further research shows that Ranked TFT has a queue_ID of 1100, so I can filter by that and skip any matches that don't match. I will make a list to store all the ranked matches data. I will also make it so the API call is made every 1.5 seconds, to make sure I don't get blocked my the API limit.

In [7]:
# Created an empty list to store my ranked match data
ranked_matches_data = []

try:
    # Loop through all the match IDs 
    for i, match_id in enumerate(match_ids):
        print(f"Fetching data for match {i+1} of {len(match_ids)}...")
        # Get the data for one match
        match_detail = tft_watcher.match.by_id(MY_REGION, match_id)

        # Check if the match is a ranked game (Queue ID: 1100)
        if match_detail['info']['queue_id'] == 1100:
            print(f"  -> Found a Ranked match. Saving it.")
            # If it is, add that match's data to the list
            ranked_matches_data.append(match_detail)
        else:
            print(f"  -> Skipping a non-ranked game.")

        # Waiting 1.5 seconds before making the next request
        time.sleep(1.5)

    print(f"\nSuccess! Found and saved {len(ranked_matches_data)} ranked matches.")

except ApiError as err:
    print(f"An API error occurred during the loop: {err}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Fetching data for match 1 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 2 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 3 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 4 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 5 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 6 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 7 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 8 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 9 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 10 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 11 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 12 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 13 of 120...
  -> Found a Ranked match. Saving it.
Fetching data for match 14 of 120.

Now I am going to save the data locally, so I don't have to rerun and wait for the API to provide me all the matches again.

In [8]:
try:
    # creating the json
    raw_json_path = os.path.join(output_folder_path, f"{output_filename_base}.json")

    # Saving the list of ranked match data to the file
    with open(raw_json_path, 'w') as f:
        json.dump(ranked_matches_data, f, indent=4)
        
    print(f"Raw JSON data for {len(ranked_matches_data)} ranked matches saved to: {raw_json_path}")
except Exception as e:
    print(f"An error occurred while saving the JSON file: {e}")

Raw JSON data for 110 ranked matches saved to: DataFolders/APIData/InitialData.json


Now to loop through the data, grab only my information instead of every player in each game, and save it to a spreadsheet.

In [12]:
try:
    # set the output patch for the ExcelSheet
    excel_output_folder = os.path.join("DataFolders", "ExcelSheets")
    os.makedirs(excel_output_folder, exist_ok=True) # Creates folder if it doesn't exist
    excel_path = os.path.join(excel_output_folder, f"{output_filename_base}.xlsx")
    
    # converting data into a table for excel
    print("Processing raw data into a table format...")
    my_processed_data = []

    # Loop through each ranked match
    for match in ranked_matches_data:
        # Loop through the 8 players in that match to find my data
        for p in match['info']['participants']:
            
            # check if the participant is me
            if p['puuid'] == my_puuid:
                
                # Once it is me, process and save my data from that match
                my_match_data = {
                    'match_id': match['metadata']['match_id'],
                    'riot_id': RIOT_ID,
                    'puuid': p['puuid'],
                    'placement': p['placement'],
                    'level': p['level'],
                    'gold_left': p['gold_left'],
                    'last_round': p['last_round'],
                    'players_eliminated': p['players_eliminated'],
                    'total_damage_to_players': p['total_damage_to_players'],
                    'time_eliminated': p['time_eliminated'],
                    'augments': ', '.join(p.get('augments', [])),
                    'traits': ', '.join([f"{trait['name']}-{trait['style']}" for trait in p.get('traits', []) if trait['tier_current'] > 0]),
                }

                # loop through each unit and add them as new columns
                for i, unit in enumerate(p.get('units', [])):
                    # create columns unit_1_id, unit_2_id, etc.
                    unit_number = i + 1
                    my_match_data[f'unit_{unit_number}_id'] = unit.get('character_id')
                    my_match_data[f'unit_{unit_number}_rarity'] = unit.get('rarity')
                    my_match_data[f'unit_{unit_number}_tier'] = unit.get('tier')
                    my_match_data[f'unit_{unit_number}_items'] = ', '.join(unit.get('itemNames', []))
                # Add the data for this match to the list
                my_processed_data.append(my_match_data)
                
                # In case I wasn't the last player but my data was found, we can stop searching this match and move onto the next
                break

    # make a pandas dataframe using the list
    df = pd.DataFrame(my_processed_data)
    df = df.fillna('')
    
    # Save the dataframe as an excel file
    df.to_excel(excel_path, index=False)
    
    print(f"\nSuccess! Your personal data for {len(df)} games has been saved to: {excel_path}")
    
except Exception as e:
    print(f"An error occurred during data processing or saving: {e}")

Processing raw data into a table format...

Success! Your personal data for 110 games has been saved to: DataFolders/ExcelSheets/InitialData.xlsx


## Excel Sheet Created
Now I can import the clean excel sheet into pandas to look at the initial data from the games I have from the set so far.

In [13]:
df = pd.read_excel('DataFolders/ExcelSheets/InitialData.xlsx')
df.head()

,match_id,riot_id,puuid,placement,level,gold_left,last_round,players_eliminated,total_damage_to_players,time_eliminated,...,unit_9_tier,unit_9_items,unit_10_id,unit_10_rarity,unit_10_tier,unit_10_items,unit_11_id,unit_11_rarity,unit_11_tier,unit_11_items
0,NA1_5317952437,saber#sadge,wIc-PVDECKl0YZaOKMwgEbF5wcGCelu79ukjEuLwd_p8nX...,5,9,6,32,0,89,1883.518188,...,1.0,"TFT_Item_RabadonsDeathcap, TFT_Item_StatikkShi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_5317929502,saber#sadge,wIc-PVDECKl0YZaOKMwgEbF5wcGCelu79ukjEuLwd_p8nX...,1,9,2,35,5,184,2045.807739,...,2.0,"TFT_Item_BlueBuff, TFT_Item_SpearOfShojin",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_5317623257,saber#sadge,wIc-PVDECKl0YZaOKMwgEbF5wcGCelu79ukjEuLwd_p8nX...,4,9,0,33,0,85,1916.759766,...,1.0,"TFT_Item_ThiefsGloves, TFT_Item_WarmogsArmor, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_5317009174,saber#sadge,wIc-PVDECKl0YZaOKMwgEbF5wcGCelu79ukjEuLwd_p8nX...,2,9,10,38,1,157,2246.447754,...,1.0,TFT_Item_LastWhisper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_5316996985,saber#sadge,wIc-PVDECKl0YZaOKMwgEbF5wcGCelu79ukjEuLwd_p8nX...,6,9,1,31,0,90,1715.448975,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 56 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 110 non-null    object 
 1   riot_id                  110 non-null    object 
 2   puuid                    110 non-null    object 
 3   placement                110 non-null    int64  
 4   level                    110 non-null    int64  
 5   gold_left                110 non-null    int64  
 6   last_round               110 non-null    int64  
 7   players_eliminated       110 non-null    int64  
 8   total_damage_to_players  110 non-null    int64  
 9   time_eliminated          110 non-null    float64
 10  augments                 0 non-null      float64
 11  traits                   110 non-null    object 
 12  unit_1_id                110 non-null    object 
 13  unit_1_rarity            110 non-null    int64  
 14  unit_1_tier              1

In [15]:
df.describe()

,placement,level,gold_left,last_round,players_eliminated,total_damage_to_players,time_eliminated,augments,unit_1_rarity,unit_1_tier,...,unit_7_rarity,unit_7_tier,unit_8_rarity,unit_8_tier,unit_9_rarity,unit_9_tier,unit_10_rarity,unit_10_tier,unit_11_rarity,unit_11_tier
count,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,0.0,110.000000,110.000000,...,109.000000,109.000000,104.000000,104.000000,74.000000,74.000000,21.000000,21.000000,5.000000,5.000000
mean,4.263636,8.609091,8.609091,32.618182,1.027273,104.963636,1878.963402,NaN,0.063636,1.854545,...,4.568807,1.733945,5.288462,1.519231,5.729730,1.513514,4.761905,1.428571,3.200000,2.000000
std,2.236795,0.691965,15.586453,4.611477,1.199840,51.384845,259.988469,NaN,0.280147,0.521754,...,1.125191,0.464324,0.962128,0.539341,1.024325,0.602353,2.022493,0.507093,2.280351,0.707107
min,1.000000,7.000000,0.000000,21.000000,0.000000,0.000000,1178.947144,NaN,0.000000,1.000000,...,2.000000,1.000000,4.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
25%,2.000000,8.000000,1.000000,30.000000,0.000000,61.500000,1704.038635,NaN,0.000000,2.000000,...,4.000000,1.000000,4.000000,1.000000,6.000000,1.000000,4.000000,1.000000,2.000000,2.000000
50%,4.000000,9.000000,1.000000,33.000000,1.000000,97.500000,1897.614441,NaN,0.000000,2.000000,...,4.000000,2.000000,6.000000,1.500000,6.000000,1.000000,6.000000,1.000000,4.000000,2.000000
75%,6.000000,9.000000,6.750000,37.000000,1.000000,142.500000,2080.402832,NaN,0.000000,2.000000,...,6.000000,2.000000,6.000000,2.000000,6.000000,2.000000,6.000000,2.000000,4.000000,2.000000
max,8.000000,10.000000,80.000000,42.000000,5.000000,208.000000,2383.494385,NaN,2.000000,3.000000,...,6.000000,3.000000,6.000000,3.000000,7.000000,4.000000,6.000000,2.000000,6.000000,3.000000
